In [1]:
from keras.layers import Input, Dense, Dropout, RepeatVector, LSTM, concatenate, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop


data = open('index.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


# image_model = Sequential()
# image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=(512,None))
# image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
# image_model.add(MaxPooling2D(pool_size=(2, 2)))
# image_model.add(Dropout(0.25))

# image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
# image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
# image_model.add(MaxPooling2D(pool_size=(2, 2)))
# image_model.add(Dropout(0.25))

# image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
# image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
# image_model.add(MaxPooling2D(pool_size=(2, 2)))
# image_model.add(Dropout(0.25))

# image_model.add(Flatten())
# image_model.add(Dense(1024, activation='relu'))
# image_model.add(Dropout(0.3))
# image_model.add(Dense(1024, activation='relu'))
# image_model.add(Dropout(0.3))

# image_model.add(RepeatVector(48))

# visual_input = Input(shape=input_shape)
# encoded_image = image_model(visual_input)

# textual_input = Input(shape=(48, output_size))

# decoder = LSTM(512, return_sequences=True)(decoder)
# decoder = LSTM(512, return_sequences=False)(decoder)
# decoder = Dense(output_size, activation='softmax')(decoder)

# model = Model(inputs=visual_input, outputs=decoder)

# optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# model.fit(self, images, partial_captions, next_words):
# self.model.fit([images, partial_captions], next_words, shuffle=False, epochs=1, batch_size=1, verbose=1)

# model.predict([image, partial_caption], verbose=0)

Using TensorFlow backend.
/Users/ewallner/.brew/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


data has 1409 characters, 58 unique.


In [3]:
print(chars)

['/', 'H', "'", 'y', '.', 'c', '=', 'h', 'M', 'u', '3', ',', 'N', 'P', 'e', '2', 'I', '>', 'L', 'f', 't', '\n', 'G', 'z', '7', 'C', '"', 'k', 'g', 'v', 'm', '4', 'S', 'd', 'i', '!', 'l', 'a', 'o', 'T', 'W', 'r', '6', '8', 's', 'O', 'w', 'R', 'p', 'F', '5', ' ', '<', 'B', '1', 'n', 'A', 'b']
